In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta

In [15]:
datastocks = pd.read_csv('data_snp_stocks.csv')
datastocks['date'] = pd.to_datetime(datastocks['date'], format = 'mixed')
#datastocks.set_index('date', inplace=True)
tickers = datastocks['Name'].unique()

In [16]:
datastocks

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [17]:
snp500 = pd.read_csv('spx.csv',)
snp500['Date'] = pd.to_datetime(snp500['Date'], format='mixed')
if 'MarketReturn' not in snp500.columns:
    snp500['MarketReturn'] = np.log(snp500['close'] / snp500['close'].shift(1))
snp500 = snp500[(snp500['date'] >= pd.to_datetime('2013-02-11')) & (snp500['date'] <= pd.to_datetime('2018-02-07'))]
snp500

In [20]:
datastocks.dtypes

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
Name              object
dtype: object

In [21]:
datastocks.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [22]:
datastocks.isna().sum()

date       0
open      11
high       8
low        8
close      0
volume     0
Name       0
dtype: int64

In [23]:
#data.sort_values(by=['Name', 'date'], inplace=True)
datastocks['LogReturn'] = datastocks.groupby('Name')['close'].transform(lambda x: np.log(x / x.shift(1)))
datastocks['LogReturn'].isna().sum()
datastocks = datastocks[datastocks['LogReturn'].notna()]
data = datastocks.merge(snp500, left_on='date', right_on='date')
data

The equation for the rolling regression:

$r_i -r_f = \alpha_i + \beta_i (R_M - r_f) + e_i$

Where:

$r_i$ is the return of the stock $i$ \
$r_f$ is the return of the risk free asset \
$\alpha_i$ is the intercept of the regression, or the excess return \
$\beta_i$ is the regression coefficient \
$R_M$ is the return of the market (log-return) \
$e_i$ is the specific risk of stock $i$


In [28]:
df_coeff = pd.DataFrame(columns=['Ticker', 'alpha', 'beta', 'r_squared', 'specific_risk', 'systematic_risk'])

In [29]:
for ticker in tickers:
    dataLoop = data[data['Name'] == ticker]

    # Drop rows with missing values
    dataLoop = dataLoop.dropna(subset=['LogReturn', 'MarketReturn'])
    if len(dataLoop) < 180:
        print(f"Skipping {ticker} due to insufficient data for rolling window.")
        continue
    for i in range(0, len(dataLoop)-180):
        y = dataLoop['LogReturn'].iloc[i:i+180].values
        X = dataLoop['MarketReturn'].iloc[i:i+180].values  # Use market returns as the independent variable
        X = sm.add_constant(X)

        model = sm.OLS(y, X)
        result = model.fit()

        df_coeff = pd.concat([df_coeff, pd.DataFrame({
            'Ticker': [ticker],
            'alpha': [result.params[0]],
            'beta': [result.params[1]],
            'r_squared': [result.rsquared],
            'specific_risk': [np.sqrt(result.mse_resid)],
            'systematic_risk': [result.params[1] ** 2 * np.var(dataLoop['MarketReturn'])]
        })], ignore_index=True)

print(df_coeff)

/var/folders/sy/ztw1lh8j1cl61nghzb_ksw2m0000gn/T/ipykernel_14666/3724646218.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_coeff = pd.concat([df_coeff, pd.DataFrame({


Skipping APTV due to insufficient data for rolling window.
Skipping BHF due to insufficient data for rolling window.
Skipping BHGE due to insufficient data for rolling window.
Skipping DWDP due to insufficient data for rolling window.
       Ticker     alpha      beta  r_squared  specific_risk  systematic_risk
0         AAL  0.001270  1.203694   0.149584       0.021111         0.000084
1         AAL  0.001335  1.200172   0.149349       0.021067         0.000083
2         AAL  0.001468  1.205443   0.151092       0.021039         0.000084
3         AAL  0.001413  1.197915   0.150615       0.020982         0.000083
4         AAL  0.001542  1.208948   0.156344       0.020737         0.000085
...       ...       ...       ...        ...            ...              ...
527906    ZTS  0.000648  0.799939   0.163620       0.008149         0.000037
527907    ZTS  0.000699  0.796320   0.160491       0.008206         0.000037
527908    ZTS  0.000691  0.799037   0.166276       0.008203         0.00

In [30]:
df_coeff.to_csv('data_snp_stocks_coefficients.csv', index=False)

In [31]:
df_coeff.groupby('Ticker').mean()

,alpha,beta,r_squared,specific_risk,systematic_risk
Ticker,,,,,
A,-0.000206,1.271370,0.403908,0.012294,0.000094
AAL,0.000306,1.515926,0.234912,0.019769,0.000141
AAP,-0.000193,0.926173,0.171729,0.016492,0.000052
AAPL,0.000484,0.962868,0.270297,0.012122,0.000059
ABBV,0.000150,1.107772,0.264815,0.013798,0.000074
...,...,...,...,...,...
XYL,0.000263,1.151586,0.406986,0.010439,0.000078
YUM,-0.000339,1.065282,0.262721,0.014893,0.000068
ZBH,-0.000045,0.984739,0.318331,0.010942,0.000057
